In [1]:
import numpy as np
import pyvista

In [2]:
import openep
from openep._datasets.openep_datasets import DATASET_2

In [3]:
case = openep.load_openep_mat(DATASET_2)
endo = case.create_mesh()
endo.point_data['Thickness'] = np.full(endo.n_points, fill_value=2, dtype=float)
endo.point_data['Colour'] = np.full(endo.n_points, fill_value=0, dtype=float)
endo.compute_normals(inplace=True)

PolyData (0x7fb2c26cbfa0)
  N Cells:	16942
  N Points:	14383
  X Bounds:	-7.584e+01, 6.374e+01
  Y Bounds:	-1.300e+02, -3.585e+01
  Z Bounds:	7.937e+01, 1.712e+02
  N Arrays:	4

In [4]:
epi = endo.copy(deep=True)
epi.points += epi.point_data['Normals'] * epi.point_data['Thickness'][:, np.newaxis]
epi.point_data['Colour'][:] = 1


In [5]:
combined = endo + epi
combined.set_active_scalars('Colour')

In [6]:
def round_up(value, nearest):
    return np.ceil(value / nearest) * nearest
def round_down(value, nearest):
    return np.floor(value / nearest) * nearest


In [7]:
slice_thickness = 10  # mm
low_values = np.asarray(combined.bounds[::2])
high_values = np.asarray(combined.bounds[1::2])

low_values = round_down(low_values, nearest=slice_thickness) - slice_thickness / 2
high_values = round_up(high_values, nearest=slice_thickness) + slice_thickness / 2


In [8]:
low_x, low_y, low_z = low_values
high_x, high_y, high_z = high_values
bounds = np.asarray([low_x, high_x, low_y, high_y, low_z, high_z])

n_slices = (1 + (high_values - low_values) / slice_thickness).astype(int)
centre = (low_values + high_values) / 2


In [51]:
axis = 0
slices = combined.slice_along_axis(
    n=n_slices[axis]-1,
    axis=axis,
    bounds=bounds,
)


In [52]:
slices

MultiBlock (0x7f9578f3e340)
  N Blocks:	13
  X Bounds:	-63.083, 43.083
  Y Bounds:	-128.693, -47.075
  Z Bounds:	99.302, 167.034

In [55]:
slices[1].points

pyvista_ndarray([[-63.08333333, -74.65703586, 129.63707987],
                 [-63.08333333, -75.24196636, 130.09186544],
                 [-63.08333333, -72.85271498, 123.68514493],
                 [-63.08333333, -72.96192855, 124.52094124],
                 [-63.08333333, -73.23269667, 128.4690227 ],
                 [-63.08333333, -73.25753175, 128.50532187],
                 [-63.08333333, -75.5663903 , 130.3954157 ],
                 [-63.08333333, -72.91636259, 121.09522025],
                 [-63.08333333, -72.75560402, 121.96885095],
                 [-63.08333333, -72.77251348, 122.76362244],
                 [-63.08333333, -73.02947113, 125.94066348],
                 [-63.08333333, -72.93380866, 126.32119403],
                 [-63.08333333, -73.41216316, 128.63192693],
                 [-63.08333333, -73.01034785, 120.81506545],
                 [-63.08333333, -72.79558118, 126.82709755],
                 [-63.08333333, -72.90796044, 127.36877448],
                 [-63.08

In [56]:
bounds

array([ -75.,   55., -145.,  -35.,   85.,  175.])

In [88]:
slices[2].plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [39]:
voxels = pyvista.voxelize(
    combined,
    check_surface=False,
    density=[1, 1, 1],
)

In [30]:
voxels.bounds

(-66.74468889713287,
 43.255311102867125,
 -132.03929526042938,
 -46.039295260429384,
 99.15148951435089,
 167.15148951435089)

In [47]:
import pymeshfix

In [49]:
epi_mf = pymeshfix.MeshFix(epi)
epi_mf.repair(joincomp=False, remove_smallest_components=True)
epi_watertight = epi_mf.mesh


93
95
108
160
67
141
84


In [50]:
endo_mf = pymeshfix.MeshFix(endo)
endo_mf.repair(joincomp=False, remove_smallest_components=True)
endo_watertight = epi_mf.mesh

93
95
108
160
67
141
84


In [51]:
combined_watertight = endo_watertight + endo_watertight

In [57]:
combined_mf = pymeshfix.MeshFix(combined)
combined_mf.repair(joincomp=True, remove_smallest_components=False)


93
84
95
108
141
67
93
322
84
95
108
141
67


In [59]:
combined_mf.mesh

PolyData,Information
N Cells,25814
N Points,12909
X Bounds,"-6.674e+01, 4.346e+01"
Y Bounds,"-1.242e+02, -4.527e+01"
Z Bounds,"9.915e+01, 1.675e+02"
N Arrays,0


In [56]:
combined_watertight

PolyData,Information
N Cells,55828
N Points,13959
X Bounds,"-6.674e+01, 4.346e+01"
Y Bounds,"-1.242e+02, -4.527e+01"
Z Bounds,"9.915e+01, 1.675e+02"
N Arrays,0


In [60]:
combined_mf.mesh.plot(opacity=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [53]:
combined_watertight.plot(opacity=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [62]:
slices[10].plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [44]:
watertight.plot(opacity=0.2).

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [61]:
slices = voxels.slice_along_axis(axis='x', contour=False, n=200)
slices = combined_watertight.slice_along_axis(axis='x', n=200)

In [68]:
pyvista.voxelize

<function pyvista.utilities.features.voxelize(mesh, density=None, check_surface=True)>

In [32]:
plotter = pyvista.Plotter()
plotter.show

<bound method Plotter.show of <pyvista.plotting.plotting.Plotter object at 0x7fb3206d7250>>

In [37]:
slices[100].plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [79]:
slices[2].plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [57]:
slices[10].plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [9]:
for s in slices:
    try:
        s.plot()
    except:
        pass

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

...

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [10]:
combined.plot(scalars='Colour')


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
slices.plot()

In [12]:
combined.save('mesh.vtk')